# Main run script

This script contains the main procedure to calculate global root zone storage capacities. 

This scripts only works in the conda environment **sr_env**. In this environment all required packages are available. If you have **not** installed and activated this environment before opening this script, you should check the installation section in the *README* file. 
 

### 1. Getting started
First, import all the required packages.

In [1]:
# import packages
import glob
from pathlib import Path
import os
import numpy as np
from datetime import datetime
from datetime import timedelta
import pandas as pd
import calendar
import geopandas as gpd
import cartopy
import matplotlib.pyplot as plt
import math
from pathos.threading import ThreadPool as Pool
from scipy.optimize import least_squares
import sklearn
from sklearn.linear_model import LinearRegression
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
import random

Here we import all the python functions defined in the scripts *f_catch_characteristics.py* and *f_preprocess_discharge.py*.

In [2]:
# import python functions
from f_catch_characteristics import *
from f_preprocess_discharge import *
from f_sr_calculation import *
from f_regression import *

### 2. Define working and data directories
Here we define the working directory, where all the scripts and output are saved.

We also define the data directory where you have the following subdirectories:

/data/forcing/*netcdf forcing files*\
/data/shapes/*catchment shapefiles*\
/data/gsim_discharge/*gsim discharge timeseries*


In [3]:
# Check current working directory (helpful when filling in work_dir below)
os.getcwd()

'/mnt/u/LSM root zone/global_sr/scripts/global_sr_module'

In [4]:
# define your script working directory
work_dir=Path("/mnt/u/LSM root zone/global_sr/")
# work_dir=Path('/tudelft.net/staff-umbrella/LSM root zone/global_sr')

# define your data directory
data_dir=Path(f'{work_dir}/data')

Here we create the output directory inside your working directory. In the remainder of this module, the same command will be used regularly to create directories.

In [5]:
# make output directory
if not os.path.exists(f'{work_dir}/output'):
    os.makedirs(f'{work_dir}/output')
out_dir = Path(f"{work_dir}/output")

### 2. GSIM discharge data
### 2.1 Catchment id lists
The GSIM discharge data contains yearly discharge files for ~30000 catchments. These files are stored here *{data_dir}/GSIM_data/GSIM_indices/TIMESERIES/yearly/* and origin from https://essd.copernicus.org/articles/10/765/2018/ and
https://essd.copernicus.org/articles/10/787/2018/.
Here we create lists of the catchment ids for later use, both with upper and lower characters. Files are saved in *out_dir/gsim/*


In [66]:
# get list of all catchment ids available in the GSIM yearly discharge timeseries data
gsim_id_list_up = []
gsim_id_list_lo = []
for filepath in glob.iglob(f'{data_dir}/GSIM_data/GSIM_indices/TIMESERIES/yearly/*'):
    f = os.path.split(filepath)[1] # remove full path
    f = f[:-5] # remove .year extension
    fl = f.lower()
    gsim_id_list_up.append(f)
    gsim_id_list_lo.append(fl)

In [67]:
if not os.path.exists(f'{work_dir}/output/gsim'):
    os.makedirs(f'{work_dir}/output/gsim')
np.savetxt(f'{out_dir}/gsim/gsim_catch_id_list_up.txt',gsim_id_list_up,fmt='%s')
np.savetxt(f'{out_dir}/gsim/gsim_catch_id_list_lo.txt',gsim_id_list_lo,fmt='%s')

### 2.2 Preprocess data 
The GSIM yearly discharge timeseries are stored in *.year* files. A detailed explanation of the column names is provided in Table 3 and 4 in https://essd.copernicus.org/articles/10/787/2018/. Here we preprocess these data into readable *.csv* files for each catchment.

The preprocessing function *preprocess_gsim_discharge* is defined in the file *f_preprocess_discharge.py*. With this function we generate for each catchment a file with the yearly discharge timeseries and a file with the specifications of the catchment.

In [6]:
# make output directories
if not os.path.exists(f'{out_dir}/gsim/timeseries'):
    os.makedirs(f'{out_dir}/gsim/timeseries')

if not os.path.exists(f'{out_dir}/gsim/timeseries_selected'):
    os.makedirs(f'{out_dir}/gsim/timeseries_selected')
    
if not os.path.exists(f'{out_dir}/gsim/characteristics'):
    os.makedirs(f'{out_dir}/gsim/characteristics')

Here we do a test run with only 5 catchments to reduce computational time. Run all catchments on Delftblue using slurm and the *run_gsim_preprocessing.py* script.

In [13]:
# load catchment ids
gsim_id_list_lo = np.loadtxt(f'{out_dir}/gsim/gsim_catch_id_list_lo.txt',dtype=str) 

# select randomly 5 catchments
# gsim_id_list_lo = random.choices(gsim_id_list_lo,k=5)
gsim_id_list_lo = gsim_id_list_lo[50:150]

# define folder with discharge timeseries data
fol_in = f'{data_dir}/GSIM_data/GSIM_indices/TIMESERIES/yearly/'

# define output folder
fol_out = f'{out_dir}/gsim/'

# make lists for parallel computation
catch_list = gsim_id_list_lo
fol_in_list = [fol_in] * len(catch_list)
fol_out_list = [fol_out] * len(catch_list)

# run function
run_function_parallel(catch_list,fol_in_list,fol_out_list)

### 2.3 select GSIM catchments
Here we select GSIM catchments that are used for further analysis. The selection criteria are as follows:
 - timeseries after 1980 contains at least 10 years of data
 - area quality high or medium

If a year has less than 250 days with data, the year is set to nan, and we remove nan years from the timeseries.
It is ok to have non-consecutive years in our timeseries.

The function *select_catchments* in *f_preprocess_discharge.py* selects catchments and stores the selected timeseries in a separate folder.
Here we do a test run with only 5 catchments to reduce computational time. Run all catchments on Delftblue using slurm and the *run_gsim_selection.py* script.

In [14]:
# load catchment ids
gsim_id_list_lo = np.loadtxt(f'{out_dir}/gsim/gsim_catch_id_list_lo.txt',dtype=str) 

# select randomly 5 catchments
# gsim_id_list_lo = random.choices(gsim_id_list_lo,k=5)
gsim_id_list_lo = gsim_id_list_lo[50:150]

# make lists for parallel computation
catch_list = gsim_id_list_lo
data_dir_list = [data_dir] * len(catch_list)
out_dir_list = [out_dir] * len(catch_list)

# run function
run_function2_parallel(data_dir_list,out_dir_list,catch_list)

Make here lists of the selected catchment ids, that are stored in the previous step in the *timeseries_selected* folder

In [15]:
gsim_id_list_up_sel = []
gsim_id_list_lo_sel = []
for filepath in glob.iglob(f'{out_dir}/gsim/timeseries_selected/*'):
    f = os.path.split(filepath)[1] # remove full path
    f = f[:-4] # remove .csv extension
    fl = f.lower()
    gsim_id_list_up_sel.append(f)
    gsim_id_list_lo_sel.append(fl)
    
np.savetxt(f'{out_dir}/gsim/gsim_catch_id_list_up_sel.txt',gsim_id_list_up_sel,fmt='%s')
np.savetxt(f'{out_dir}/gsim/gsim_catch_id_list_lo_sel.txt',gsim_id_list_lo_sel,fmt='%s')

### 3. Organize data

**Add Australia CAMELS catchments**\
The CAMELS-AU data is added to GSIM to get a better representation of climate regions. The discharge data is stored in /data/CAMELS_AUS/ and is processed using *preprocess_q_aus.py*
Output is stored in /output/camels-aus/

Here we assume all catchments are ok for Australia. 
- Copy all files in /output/camels_aus/ to /output/gsim/timeseries_selected/ (do manually) folder to complete the GSIM dataset, and move one folder up in output/
- And we need to make a new list with selected catchments including australia - see below.
- Copy all shapefiles of the selected GSIM and CAMELS-AUS catchments into /output/selected_shapes/
- Copy all selected q_timeseries in /output/gsim/timeseries_selected to /output/q_timeseries_selected/

In [10]:
gsim_id_aus_list_up_sel = []
gsim_id_aus_list_lo_sel = []
for filepath in glob.iglob(f'{out_dir}/q_timeseries_selected/*'):
    f = os.path.split(filepath)[1] # remove full path
    f = f[:-4] # remove .csv extension
    fl = f.lower()
    gsim_id_aus_list_up_sel.append(f)
    gsim_id_aus_list_lo_sel.append(fl)
    
np.savetxt(f'{out_dir}/gsim_aus_catch_id_list_up_sel.txt',gsim_id_aus_list_up_sel,fmt='%s')
np.savetxt(f'{out_dir}/gsim_aus_catch_id_list_lo_sel.txt',gsim_id_aus_list_lo_sel,fmt='%s')

In [12]:
catch_list = np.loadtxt(f'{out_dir}/gsim_aus_catch_id_list_up_sel.txt',dtype=str)
print(catch_list)
print('The amount of selected catchments is:',len(catch_list))

['102101A' 'ar_0000001' 'ar_0000005' ... 'G9030124' 'G9030250' 'G9070142']
The amount of selected catchments is: 8658


### 5. From gridded data to catchment timeseries
For this step go to the notebook *run_script_grid_to_catchments*. This part is run in another notebook. The output data of this script can be found in:
- */output/p_gswp_timeseries_selected_catchments*
- */output/ep_gswp_timeseries_selected_catchments*
- */output/tas_gswp_timeseries_selected_catchments*

In [13]:
# TO DO HERE - POSTPROCESS FORCING TIMESERIES AND USE FORMAT FROM RUN_SCRIPT_GRID_TO_CATCHMENTS

### 6. Google earth engine for catchment characteristics
For this step go to the notebook *run_script_earthengine*. This part is run in another notebook. The output data of this script can be found in *work_dir/output/earth_engine_timeseries*.\\

- Treecover output is stored in */output/treecover/treecover_selected_shapes.csv*

### 7. Catchment descriptor variables
For the global root zone storage capacity estimation, we need to calculate catchment descriptor variables. These descriptors can be climatological variables (e.g. mean precipitation (p_mean); seasonality of precipitation (si_p); timelag between maximum P and Ep (phi)) or landscape variables (e.g. mean treecover (tc); mean elevation (h_mean)). A detailed list of all the descriptors considered is provided here xxxxx.\
To calculate the catchment descriptor variables we use the *catch_characteristics* function from the *f_catch_characteristics.py* file. In this function you specify the variables of interest, the catchment ID and your in- and output folders. Then, based on all the timeseries you have generated in the preceding codes it will return a table with the catchment descriptor variables for all your catchments (that is saved as csv in your *work_dir/catchment_characteristics.csv*).

In [ ]:
# define in and output folder
fol_in=f'{work_dir}/output/'
fol_out=f'{work_dir}/output/'

# define variables of interest
var=['p_mean','ep_mean','q_mean','t_mean','ai','si_p','si_ep','phi','tc','ntc','nonveg']

# run catch_characteristics (defined in f_catch_characteristics.py) for the catchments in your catch_id_list
catch_characteristics(var, catch_id_list, fol_in, fol_out)

In [ ]:
# combine catchment geometries in single shapefile
shape_dir = f'{data_dir}/shapes/'
out_dir = f'{work_dir}/output'
geo_catchments(shape_dir,out_dir)

## check also for WB and for AREA (<10000km2) - see catchment_waterbalance.ipynb

### 8. Calculate root zone storage capacity
Here we calculate the catchment root zone storage capacity (Sr) based on catchment water balances. First, catchment root zone storage deficits (Sd) are computed as the cumulative difference between P and Et (transpiration). The result of one catchment is visualised in a figure. Second, the Sr is then calculated based on an extreme value analysis of the storage deficits for different return periods. A detailed description of this method can be found here xxxxxx.

Here we use the *run_sd_calculation* and *run_sr_calculation* functions from the *f_sr_calculation* file. The Sd result of one catchment is visualised using *plot_sd*. The Sr results are merged using the *merge_sr* function and visualised using the *plot_sr* function. The output of both storage deficit and Sr calculations are saved in your *work_dir/output/sr_calculation*.


In [ ]:
# make output directories
if not os.path.exists(f'{work_dir}/output/sr_calculation/sd_catchments'):
    os.makedirs(f'{work_dir}/output/sr_calculation/sd_catchments')
    
if not os.path.exists(f'{work_dir}/output/sr_calculation/sr_catchments'):
    os.makedirs(f'{work_dir}/output/sr_calculation/sr_catchments')

Calculate storage deficits using the *run_sd_calculation* function from *f_sr_calculation*

In [ ]:
# define directories
pep_dir = f'{work_dir}/output/forcing_timeseries/processed/daily'
q_dir = f'{work_dir}/output/discharge/timeseries'
out_dir = f'{work_dir}/output/sr_calculation/sd_catchments'

# run sd calculation for all catchments in catch_id_list
for catch_id in catch_id_list:
    run_sd_calculation(catch_id, pep_dir, q_dir, out_dir)
    
#comRuud: print to screen what is being created (for some reason I did not get an Sd for the US catchment, but no error!)
# Fransje: this is correct, incorrect water balance in us catchment -> no sd calculated. add flag or so in table??

In [ ]:
# plot sd example - use the first catchment from catch_id_list
sd_dir = f'{work_dir}/output/sr_calculation/sd_catchments'
plot_sd(catch_id_list[0], sd_dir)

Calculate Sr using the *run_sr_calculation* function from *f_sr_calculation* and merge the catchment Sr values into one dataframe with *merge_sr_catchments*. The functions return a table with the catchment Sr values for the different return periods.

In [ ]:
# define directories
sd_dir = f'{work_dir}/output/sr_calculation/sd_catchments'
out_dir = f'{work_dir}/output/sr_calculation/sr_catchments'

# define return periods
rp_array = [2,3,5,10,20,30,40,50,60]

# run sr calculation for all catchments in catch_id_list
for catch_id in catch_id_list:
    run_sr_calculation(catch_id, rp_array, sd_dir, out_dir)
    
# merge catchment sr dataframes into one dataframe
sr_dir = f'{work_dir}/output/sr_calculation/sr_catchments'
out_dir = f'{work_dir}/output/sr_calculation/'
merge_sr_catchments(sr_dir,out_dir)

#comRuud: multiple Sr have been created (for different return periods? But values are the same, is that correct?) print to screen and tell what it should look like

Mapping Sr using the *plot_sr* function from *f_sr_calculation*

In [ ]:
sr_file = f'{work_dir}/output/sr_calculation/sr_all_catchments.csv'
shp_file = f'{work_dir}/output/geo_catchments.shp'
rp=20
plot_sr(shp_file,sr_file,rp)

### 9. Regression

**move this to different script? to separate 'preprocessing' and 'analysis'?**

Here we run the linear regression model to predict the catchment Sr values based on the descriptor parameters. We use the *f_regression* function to calculate the linear regression parameters for the considered catchments.
We use the treecover data to separate the regression for high and low vegetation, the threshold values for tree cover (tc), non tree cover (ntc) and no-vegetation (nonveg) define this separation.

The output is a figure showing the estimated (step 8) and predicted (from regression) Sr values and a table with the regression parameter values, some statistics for the regression performance and the threshold values for tree cover. 

In [ ]:
# read the catchment characteristics and sr tables
cc_df = pd.read_csv(f'{work_dir}/output/catchment_characteristics.csv',index_col=0)
sr_df = pd.read_csv(f'{work_dir}/output/sr_calculation/sr_all_catchments.csv',index_col=0)

# define the descriptor variables
dpar = ['p_mean','ep_mean','t_mean','si_p']

# return period of Sr estimate
rp = 20

# define the vegetation thresholds for the regression
tc_th, ntc_th, nonveg_th = 10, 0, 0

# run the regression (r_regression in f_regression.py)
run_regression(cc_df, sr_df, dpar, rp, tc_th, ntc_th, nonveg_th)